In [2]:
from utils import DocumentDB
db = DocumentDB()
print("loaded")

loaded


In [3]:
from src import SemanticIndex, BM25Index

semantic_idx = SemanticIndex()
bm25_idx = BM25Index()

print("loaded")

/home/dhairya.dalal/.conda/envs/nlp-env/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
loaded


In [ ]:
from sentence_transformers import CrossEncoder
ranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [10]:
doc_ids = [i[0] for i in qrel["1"].items()]
texts = db.batch_docno_lookup(doc_ids)

query = topics[0]["title"]
pairs = list(zip([query] * len(texts), texts))

In [11]:
preds = ranker.predict(pairs)

In [17]:
pos_ctr = 0
neg_ctr = 0
for i, v in enumerate(qrel["1"].items()):
    print(v[0], v[1], preds[i])
    if v[1] == 0:
        neg_ctr += 1
    else:
        pos_ctr += 1

1101004_foreign_story_13015055.utf8 1 -5.9750423
1101002_foreign_story_13006291.utf8 1 3.236979
1100716_foreign_index.utf8 0 -1.0389109
1100716_foreign_story_12689429.utf8 0 -0.79132426
1100710_foreign_story_12667785.utf8 0 0.07188113
1100616_foreign_story_12570790.utf8 1 3.850542
1100510_opinion_story_12411827.utf8 1 1.4522578
1100125_foreign_story_12025749.utf8 1 2.164837
1091226_foreign_story_11910177.utf8 0 1.5910602
1091130_foreign_index.utf8 0 -0.016197078
1090915_foreign_index.utf8 0 0.08885631
1090915_foreign_story_11495978.utf8 1 1.077204
1090914_foreign_story_11490372.utf8 0 5.2121363
1090724_foreign_story_11275868.utf8 0 4.704734
1090604_foreign_index.utf8 1 -0.9948252
1090604_foreign_story_11060253.utf8 1 0.7138958
1090428_foreign_index.utf8 0 1.9561602
1090428_foreign_story_10886383.utf8 0 5.082822
1090118_foreign_story_10406051.utf8 0 1.2538654
1081110_frontpage_story_10088589.utf8 1 0.7490781
1080803_foreign_index.utf8 0 0.7565006
1080710_foreign_story_9529901.utf8 0 2.0

In [22]:
print(pos_ctr, neg_ctr)
run = {"1": dict(zip(doc_ids, preds.tolist()))}
print_cum_stats(run)

92 111
Aggregate results
Average MAP:  0.4934683496031204
Average P_5:  0.0
Average NDCG:  0.7914650633499363


In [24]:
p2 = [max(val, 0) for val in preds.tolist()]
run = {"1": dict(zip(doc_ids, p2))}
print_cum_stats(run)

Aggregate results
Average MAP:  0.49188967040693543
Average P_5:  0.0
Average NDCG:  0.7910598955734557


In [40]:
from tqdm.notebook import tqdm

run = {}
for topic in tqdm(topics):
    number = topic["number"]
    query = topic["title"]

    extracted_ids = [k for k in qrel[number].keys()]

    doc_ids = []
    for id in extracted_ids:
        try:
            db.lookup_docno(id)
            doc_ids.append(id)
        except:
            continue

    texts = db.batch_docno_lookup(doc_ids)
    pairs = list(zip([query] * len(texts), texts))

    scores = ranker.predict(pairs)
    scores = scores.tolist()
    
    doc_scores = dict(zip(doc_ids, scores))
    run[number] = doc_scores
print_cum_stats(run)


Aggregate results
Average MAP:  0.5326817223959618
Average P_5:  0.5799999999999998
Average NDCG:  0.8269792734077868


## Base CrossEncoder training w/ BCE Loss


Average MAP:  0.6012651069594099

Average P_5:  0.72

Average NDCG:  0.8600546413378876


## Constrastive Loss 
Aggregate results

Average MAP:  0.5993319614460655

Average P_5:  0.72

Average NDCG:  0.8591562808100097



In [46]:
# Base loss
from sentence_transformers import SentencesDataset, losses
from sentence_transformers.readers import InputExample

examples = []

for topic in topics:
    gold = qrel[topic["number"]].items()
    query = topic["title"].strip()

    for item in gold:
        try:
            doc = db.lookup_docno(item[0])
            examples.append(InputExample(texts=[query, doc], label=item[1]))
        except:
            continue
print("finished")

finished


In [50]:
from torch.utils.data import DataLoader
train_dataset = SentencesDataset(examples, ranker)
train_dl = DataLoader(train_dataset, shuffle=True, batch_size = 24)
ranker.fit(train_dataloader=train_dl, epochs=5,  output_path="ranker/base")

#train_loss =  losses.MultipleNegativesRankingLoss(model=ranker)

RuntimeError: [enforce fail at CPUAllocator.cpp:65] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 402653184 bytes. Error code 12 (Cannot allocate memory)

In [120]:
ranker.fit(train_dataloader=train_dl, epochs=10)

In [122]:
doc_ids = [i[0] for i in qrel["20"].items()]
texts = db.batch_docno_lookup(doc_ids)

In [127]:
query = topics[4]["title"]

pairs = list(zip([query] * len(texts), texts))

In [109]:
ranker.predict(pairs)

array([-1.2180766 ,  2.380508  ,  3.7832723 , -0.6131114 ,  3.4686193 ,
       -1.469696  ,  1.4555517 ,  1.4948552 , -1.248918  , -7.9772053 ,
        4.381545  ,  0.86039066,  0.36753726,  4.004533  ,  1.6861217 ,
        5.4812374 ,  4.6820188 ,  3.05331   ,  3.398644  , -4.625394  ,
        3.3283222 ,  1.7538052 ,  1.1144562 ,  2.1483555 ,  3.0440204 ,
        1.3770123 ,  3.139276  ,  5.4170923 ,  3.1966743 ,  1.1454346 ,
        3.4415848 , -0.05710527,  0.9272184 ,  0.7655416 ], dtype=float32)

In [128]:
ranker.predict(pairs)

array([ 0.47619975,  1.9613491 ,  3.4600346 ,  2.047238  ,  5.735491  ,
        2.794967  ,  5.7426586 ,  6.668082  ,  5.684595  , -2.1745458 ,
        4.680272  ,  6.2307715 ,  3.60517   ,  7.800994  ,  2.0835526 ,
        1.2318438 ,  2.157736  , -0.14282535, -0.01890556], dtype=float32)

In [126]:
topics

[{'number': '16',
  'title': 'Telecom minister A. Raja resignation',
  'narrative': 'Documents retrieved should contain information about the accusations against A. Raja in the 2G Spectrum scam. Documents that talk about his criticisms for allocating 4.4MHz of spectrum  a scarce resource  at the 2001 price of Rs 1,651 crore through a first come, first served policy rather than auctions; by compromising on the market value of spectrum, the government is estimated to have lost revenues worth around Rs 50,000 crore; new licensees are signing equity deals with foreign players in which the value of spectrum has risen sevenfold, would be considered as relevant.\n'},
 {'number': '17',
  'title': 'Brinda Karat accused Swami Ramdev',
  'narrative': "A relevant document should contain information about complaints made by Brinda Karat alleging that Ramdev has been selling medicines containing animal parts, thus violating licensing and labeling regulations laid down in India's Drug and Cosmetics A